In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import *
from tensorflow import keras
from tensorflow.keras.optimizers import *
from tensorflow.keras import layers

In [2]:
# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

import gc 
gc.collect()

22

In [3]:
import warnings

warnings.simplefilter(action = "ignore", category = RuntimeWarning)

In [4]:
time = pd.to_datetime('now').strftime("%Y%b%d%H%M%S")
model_name = f'unetv3_test4_{time}'
train = pd.read_csv(r"G:\My Drive\2021fall\NRE5545\modules\project\data\3. processed_images\train.csv")
validation = pd.read_csv(r"G:\My Drive\2021fall\NRE5545\modules\project\data\3. processed_images\validation.csv")

train_x = np.array([np.load(f) for f in train['x'].tolist()[:]])
train_y = np.array([np.load(f) for f in train['y'].tolist()[:]])
val_x = np.array([np.load(f) for f in validation['x'].tolist()[:]])
val_y = np.array([np.load(f) for f in validation['y'].tolist()[:]])
# train_x = np.random.random((30,320,320,6))
# temp = np.random.random((30,320,320))
# train_y = np.array([((temp<=0.5)*1),((temp>0.5)*1)])
# train_y = np.moveaxis(train_y, 0, -1)

# val_x = np.random.random((10,320,320,6))
# temp = np.random.random((10,320,320))
# val_y = np.array([((temp<=0.5)*1),((temp>0.5)*1)])
# val_y = np.moveaxis(val_y, 0, -1)

In [5]:
import cv2
import numpy as np
import scipy
def sharpening(temp):
    kernel = np.array([[0, -1, 0],
                       [-1, 5,-1],
                       [0, -1, 0]])
    image_sharp = cv2.filter2D(src=temp, ddepth=-1, kernel=kernel)
    return image_sharp

def edge(temp):
    kernel = np.array([[-1, -2, -1],
                       [-2, 12, -2],
                       [-1, -2, -1]])
    image_sharp = cv2.filter2D(src=temp, ddepth=-1, kernel=kernel)
    return image_sharp

def pca_transform(MB_img):
    img_shape = c,r,n_bands = MB_img.shape
    MB_matrix = np.zeros((c*r,n_bands))
    for i in range(n_bands):
        MB_array = MB_img[:,:,i].flatten()  # covert 2d to 1d array 
        MB_arrayStd = (MB_array - MB_array.mean())/MB_array.std()  
        MB_matrix[:,i] = MB_arrayStd

    cov = np.cov(MB_matrix.transpose())

    # Eigen Values
    EigVal,EigVec = np.linalg.eig(cov)

    order = EigVal.argsort()[::-1]
    EigVal = EigVal[order]
    EigVec = EigVec[:,order]

    #Projecting data on Eigen vector directions resulting to Principal Components 
    PC = np.matmul(MB_matrix,EigVec)   #cross product

    # Rearranging 1-d arrays to 2-d arrays of image size

    PC_2d = np.zeros((img_shape[0],img_shape[1],n_bands))
    for i in range(n_bands):
        PC_2d[:,:,i] = PC[:,i].reshape(-1,img_shape[1])

    # normalizing between 0 to 255
    PC_2d_Norm = np.zeros((img_shape[0],img_shape[1],n_bands))
    for i in range(n_bands):
        PC_2d_Norm[:,:,i] = cv2.normalize(PC_2d[:,:,i],
                        np.zeros(img_shape),0,255 ,cv2.NORM_MINMAX)
    return PC_2d_Norm

def moving_average(temp, window = 3):
    temp  = scipy.ndimage.uniform_filter(temp, size=window, mode='constant')
    return temp

def normalize(temp):
    for i in range(temp.shape[-1]):
        temp_norm = temp[:,:,i]
        temp_norm = temp_norm.astype(np.float32) - np.mean(temp_norm)
        
        temptemp = max(temp_norm.min(), temp_norm.max(), key=abs)
        temp_norm= temp_norm.astype(np.float32)/temptemp
        temp[:,:,i] = temp_norm
    return np.array(temp)

def process_img(temp):
#     temp1 = np.expand_dims(cv2.cvtColor(sharpening(temp)[:,:,3:6], cv2.COLOR_BGR2GRAY), axis = -1)
    temp1 = sharpening(temp)[:,:,:]
#     print(temp1.shape)
    temp2 = np.expand_dims(cv2.cvtColor(edge(temp)[:,:,2:5], cv2.COLOR_BGR2GRAY), axis = -1)
    temp3 = pca_transform(temp)[:,:,:4]
    temp4 = moving_average(temp)[:,:,:3]
    temp = np.concatenate([temp[:,:,:],temp1,temp2,temp3,temp4],axis = -1)
    return normalize(temp)
    

In [6]:
train_x = np.array([process_img(each) for each in train_x])
val_x = np.array([process_img(each) for each in val_x])

In [7]:
print(train_x.shape, train_y.shape, val_x.shape, val_y.shape)

(300, 320, 320, 20) (300, 320, 320, 2) (123, 320, 320, 20) (123, 320, 320, 2)


In [8]:
def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size)

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model

In [9]:
def get_model2(img_size, num_classes, use_channels = 6):
    inputs = Input(shape=img_size)

#     s = Lambda(lambda x: x[:,:,:,:use_channels]) (inputs)
    s = inputs

    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (s)
    c1 = Dropout(0.1) (c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (p1)
    c2 = Dropout(0.1) (c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (p2)
    c3 = Dropout(0.2) (c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (p3)
    c4 = Dropout(0.2) (c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (p4)
    c5 = Dropout(0.3) (c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (u6)
    c6 = Dropout(0.2) (c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (u7)
    c7 = Dropout(0.2) (c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (u8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (u9)
    c9 = Dropout(0.1) (c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='VarianceScaling', padding='same') (c9)

    outputs = Conv2D(num_classes, (1, 1), activation='sigmoid') (c9)

    model = keras.Model(inputs=[inputs], outputs=[outputs])
    return model

In [10]:
# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()
img_size = train_x[0].shape#(320,320,6)
num_classes = train_y[0].shape[-1]#1
# Build model
# model2 = get_model(img_size, num_classes)
model2 = get_model2(img_size, num_classes)
model2.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 320, 320, 20 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 320, 320, 16) 2896        input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 320, 320, 16) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 320, 320, 16) 2320        dropout[0]

In [11]:
model_name

'unetv3_test4_2021Dec03202540'

In [12]:
# Configure the model for training.
# We use the "sparse" version of categorical_crossentropy
# because our target data is integers.sparse_categorical_crossentropy
# model2.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model2.compile(
    optimizer=RMSprop(lr = 0.0001, rho=0.5, momentum=0.1, epsilon=1e-06), 
    loss="categorical_crossentropy", 
    metrics=['accuracy'])

callbacks = [
    keras.callbacks.ModelCheckpoint(f"{model_name}.h5", save_best_only=True)
]

# Train the model, doing validation at the end of each epoch.
epochs = 300
history = model2.fit(train_x,train_y,batch_size = 10, epochs=epochs, validation_data=(val_x,val_y), callbacks=callbacks, use_multiprocessing = True)
pd.DataFrame(history.history).to_csv(f'{model_name}.csv', index=False)

Train on 300 samples, validate on 123 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/300
300/300 [==============================] - 10s 34ms/sample - loss: 0.6822 - acc: 0.6795 - val_loss: 0.6685 - val_acc: 0.7797
Epoch 2/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.6541 - acc: 0.7637 - val_loss: 0.6248 - val_acc: 0.8165
Epoch 3/300
300/300 [==============================] - 7s 23ms/sample - loss: 0.6043 - acc: 0.7997 - val_loss: 0.5597 - val_acc: 0.8385
Epoch 4/300
300/300 [==============================] - 7s 23ms/sample - loss: 0.5491 - acc: 0.8183 - val_loss: 0.4974 - val_acc: 0.8461
Epoch 5/300
300/300 [==============================] - 8s 26ms/sample - loss: 0.5038 - acc: 0.8236 - val_loss: 0.4427 - val_acc: 0.8471
Epoch 6/300
300/300 [==============================] - 8s 26ms/sample - loss: 0.4649 - acc: 0.8234 - val_loss: 0.3926 - val_acc: 0.8472
Epoch 7/300
300/300 [======================

Epoch 58/300
300/300 [==============================] - 8s 25ms/sample - loss: 0.3327 - acc: 0.8630 - val_loss: 0.2779 - val_acc: 0.8788
Epoch 59/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.3305 - acc: 0.8645 - val_loss: 0.2821 - val_acc: 0.8777
Epoch 60/300
300/300 [==============================] - 8s 25ms/sample - loss: 0.3264 - acc: 0.8646 - val_loss: 0.2762 - val_acc: 0.8800
Epoch 61/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.3281 - acc: 0.8647 - val_loss: 0.2765 - val_acc: 0.8796
Epoch 62/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.3262 - acc: 0.8659 - val_loss: 0.2790 - val_acc: 0.8800
Epoch 63/300
300/300 [==============================] - 8s 25ms/sample - loss: 0.3256 - acc: 0.8658 - val_loss: 0.2724 - val_acc: 0.8818
Epoch 64/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.3241 - acc: 0.8664 - val_loss: 0.2746 - val_acc: 0.8818
Epoch 65/300
300/300 [===================

300/300 [==============================] - 8s 26ms/sample - loss: 0.2505 - acc: 0.9004 - val_loss: 0.2503 - val_acc: 0.8995
Epoch 177/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.2426 - acc: 0.9013 - val_loss: 0.2863 - val_acc: 0.8842
Epoch 178/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.2463 - acc: 0.9014 - val_loss: 0.2682 - val_acc: 0.8919
Epoch 179/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.2422 - acc: 0.9010 - val_loss: 0.2519 - val_acc: 0.8988
Epoch 180/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.2432 - acc: 0.9021 - val_loss: 0.2841 - val_acc: 0.8840
Epoch 181/300
300/300 [==============================] - 8s 25ms/sample - loss: 0.2470 - acc: 0.9014 - val_loss: 0.2480 - val_acc: 0.9009
Epoch 182/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.2380 - acc: 0.9036 - val_loss: 0.2687 - val_acc: 0.8936
Epoch 183/300
300/300 [=========================

300/300 [==============================] - 7s 22ms/sample - loss: 0.2138 - acc: 0.9114 - val_loss: 0.3446 - val_acc: 0.8644
Epoch 295/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.2153 - acc: 0.9090 - val_loss: 0.2956 - val_acc: 0.8800
Epoch 296/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.2091 - acc: 0.9136 - val_loss: 0.2841 - val_acc: 0.8904
Epoch 297/300
300/300 [==============================] - 7s 23ms/sample - loss: 0.2139 - acc: 0.9111 - val_loss: 0.3011 - val_acc: 0.8811
Epoch 298/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.2068 - acc: 0.9137 - val_loss: 0.2569 - val_acc: 0.9025
Epoch 299/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.2108 - acc: 0.9125 - val_loss: 0.2527 - val_acc: 0.9036
Epoch 300/300
300/300 [==============================] - 7s 22ms/sample - loss: 0.2165 - acc: 0.9126 - val_loss: 0.2499 - val_acc: 0.9036
